In [1]:
import scipy.io as scipy
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from time import time
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.gaussian_process import GaussianProcessRegressor

In [2]:
mat1 = scipy.loadmat('Proj2FeatVecsSet1.mat')
mat2 = scipy.loadmat('Proj2TargetOutputsSet1.mat')
data_set =  mat1['Proj2FeatVecsSet1']
data_target = mat2['Proj2TargetOutputsSet1']

print data_set.shape

(25000L, 60L)


In [3]:
pca = PCA(n_components=9)
reduced_data = pca.fit_transform(data_set)
print reduced_data.shape

(25000L, 9L)


In [4]:
print data_target.shape

(25000L, 5L)


In [5]:
number_labels = []
for ars in data_target:
    if np.all(ars == [1, -1, -1, -1, -1]):
        ars = 1
        number_labels.append(ars)
    elif np.all(ars == [-1, 1, -1, -1, -1]):
        ars = 2
        number_labels.append(ars)
    elif np.all(ars == [-1, -1, 1, -1, -1]):
        ars = 3
        number_labels.append(ars)
    elif np.all(ars == [-1, -1, -1, 1, -1]):
        ars = 4
        number_labels.append(ars)
    elif np.all(ars == [-1, -1, -1, -1, 1]):
        ars = 5
        number_labels.append(ars)

number_labels = np.asarray(number_labels)

print number_labels.shape

(25000L,)


In [6]:
print number_labels

[1 1 1 ... 5 5 5]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    reduced_data, number_labels, test_size=0.20, stratify = data_target)#, random_state=39)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape


(20000L, 9L)
(5000L, 9L)
(20000L,)
(5000L,)


In [8]:
# print 'started training'
# t0 = time()
# clf = GaussianProcessClassifier(multi_class='one_vs_rest')
# clf.fit(X_test,y_test)
# print 'ok'
# print time()-t0

In [9]:
# y_pred = clf.predict(X_train)
# print y_pred.shape
# print(classification_report(y_train, y_pred))
# print(confusion_matrix(y_train, y_pred))

In [10]:
# X_train_1 = X_train[:4999]
# X_train_2 = X_train[5000:9999]
# X_train_3 = X_train[10000:14999]
# X_train_4 = X_train[15000:20000]
# y_train_1 = y_train[:4999]
# y_train_2 = y_train[5000:9999]
# y_train_3 = y_train[10000:14999]
# y_train_4 = y_train[15000:20000]

In [11]:
from sklearn.multiclass import OneVsOneClassifier

In [12]:
# print("Fitting the classifier to the training set")
# t0 = time()
# clf = OneVsOneClassifier(GaussianProcessClassifier())#kernel = kernel, alpha = 1e-10)#kernel=kernel) # Uses RBF as default
# clf.fit(X_train_1, y_train_1)
# print("done in %0.3fs" % (time() - t0))

In [13]:
# print("Predicting class on the test set")
# t0 = time()
# y_pred = clf.predict(X_test)
# print("done in %0.3fs" % (time() - t0))
# print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))

In [40]:
from sklearn.model_selection import KFold
import glob
import os
# Not sure if this is the correct type of function we need
def TrainMyClassifier(XEstimate, YEstimate, XValidate, TrainMyClassifierParameters):
    from sklearn.svm import SVC
#     from skrvm import RVC
    from sklearn.gaussian_process import GaussianProcessClassifier
    from sklearn.multiclass import OneVsOneClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.gaussian_process.kernels import RBF
    from time import time
    t0 = time()
    # Paramaters should have this shape in order for it to work ==>  Parameters = {'C': [1e3, 1e4, 1e5], 'gamma': [0.001, 0.01, 0.1] }
    if TrainMyClassifierParameters[1] == 'SVM':
        # ################################################
        # Train a SVM classification model
        print("Fitting the classifier to the training set")
        param_grid = TrainMyClassifierParameters[0]
        clf = OneVsOneClassifier(GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid))
        clf = clf.fit(XEstimate, YEstimate)
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores
    elif TrainMyClassifierParameters[1] == 'RVM':
        # #############################################################################
        # Train a RVM classification model
        print("Fitting the classifier to the training set")
        t0 = time()
        param_grid = TrainMyClassifierParameters[0]
        clf = OneVsOneClassifier(GridSearchCV(RVC(kernel='rbf',n_iter=1), param_grid))
        clf.fit(XEstimate, YEstimate)
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores
    elif TrainMyClassifierParameters[1] == 'GP':
       # #############################################################################
        # Train a GP classification model
        print("Fitting the classifier to the training set")
        t0 = time()
        param_grid = TrainMyClassifierParameters[0]['length_scale']
        k_rbf = RBF(length_scale=param_grid)
        clf = OneVsOneClassifier(GaussianProcessClassifier(kernel = k_rbf))
        clf.fit(X_train_1, y_train_1)
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores
    else:
        print("Incorrect type of algorithm, please use only one of the supported classifiers SVM, RVM, GP")

def MyCrossValidate(XTrain,YTrain2,Nf,Algorithm):
    dict = {} 
    
    pca = PCA(n_components=9)
    reduced_data = pca.fit_transform(XTrain)
    #print reduced_data.shape
    kf = KFold(n_splits=Nf)
    kf.get_n_splits(XTrain)
    EstParameters = []
    EstConfMatrices = []
    ConfMatrix=[]
    YTrain = []
    i=0        
    for train_index, test_index in kf.split(XTrain):
        # print("TRAIN:", train_index, "TEST:", test_index)
        XEst1 = XTrain[train_index]
        YEst1 = YTrain2[train_index]      
        XValid = XTrain[test_index]
        YValid = YTrain2[test_index]          
        XEst = XEst1[:4000]
        YEst = YEst1[:4000]
        TrainMyClassifierParameters = []
        TrainMyClassifierParameters.append({'C': [1e3], 'gamma': [0.001] })
        TrainMyClassifierParameters.append(Algorithm)
        y_pred1, scores1 = TrainMyClassifier(XEst,YEst,XValid,TrainMyClassifierParameters)
        TrainMyClassifierParameters[0] = {'C': [1e4], 'gamma': [0.01]}
        y_pred2, scores2 = TrainMyClassifier(XEst,YEst,XValid,TrainMyClassifierParameters)
        TrainMyClassifierParameters[0] = {'C': [1e5], 'gamma': [0.1]}
        y_pred3, scores3 = TrainMyClassifier(XEst,YEst,XValid,TrainMyClassifierParameters)
        if max(scores1,scores2,scores3)==scores1:
            y_pred = y_pred1
            dict[i]= {'scores':[scores1,scores2,scores3],'C': [1e3], 'gamma': [0.001]}
        elif max(scores1,scores2,scores3)==scores2:
            y_pred = y_pred2
            dict[i]= {'scores':[scores1,scores2,scores3],'C': [1e4], 'gamma': [0.01]}
        elif max(scores1,scores2,scores3)==scores3:
            y_pred = y_pred3  
            dict[i]= {'scores':[scores1,scores2,scores3],'C': [1e5], 'gamma': [0.1]}
        confMatrix = MyConfusionMatrix(y_pred,YValid)
        EstConfMatrices.append(confMatrix)
#         EstParameters.append(params)
        YTrain.append(y_pred)
        i=i+1
#         y_pred, params = TrainMyClassifier(XEst,YEst,XValid,Algorithm,{'C': [1], 'gamma': [1] })
#         confMatrix = MyConfusionMatrix(y_pred,YValid)
#         EstConfMatrices.append(confMatrix)
#         EstParameters.append(params)
#         YTrain.append(y_pred)
    np.save(Algorithm+'.npy',dict)
    return YTrain,EstParameters, EstConfMatrices,ConfMatrix
def MyConfusionMatrix(Y,ClassNames):
        return confusion_matrix(ClassNames,Y)
       
YTrain,EstParameters, EstConfMatrices,ConfMatrix = MyCrossValidate(X_train,y_train,5,'SVM')
print EstConfMatrices
        
        

Fitting the classifier to the training set
done in 1.808s
Fitting the classifier to the training set
done in 1.540s
Fitting the classifier to the training set
done in 9.229s
Fitting the classifier to the training set
done in 1.804s
Fitting the classifier to the training set
done in 1.473s
Fitting the classifier to the training set
done in 10.310s
Fitting the classifier to the training set
done in 1.751s
Fitting the classifier to the training set
done in 1.316s
Fitting the classifier to the training set
done in 10.371s
Fitting the classifier to the training set
done in 1.640s
Fitting the classifier to the training set
done in 1.478s
Fitting the classifier to the training set
done in 10.580s
Fitting the classifier to the training set
done in 1.770s
Fitting the classifier to the training set
done in 1.486s
Fitting the classifier to the training set
done in 10.628s
[array([[772,   5,   7,   3,   1],
       [  5, 762,   5,  60,   1],
       [  5,   4, 716,   5,  25],
       [  5,  54,  11, 

In [23]:
# TrainMyClassifierParameters = []
# TrainMyClassifierParameters.append({'C': [1], 'gamma': [1] })
# TrainMyClassifierParameters.append('SVM')
# TrainMyClassifierParameters[0] = {'C': [2], 'gamma': [2]}
# print TrainMyClassifierParameters[0]


{'C': [2], 'gamma': [2]}


In [41]:
read_dictionary = np.load('SVM.npy').item()
print read_dictionary

{0: {'C': [100000.0], 'gamma': [0.1], 'scores': [0.926, 0.942, 0.976]}, 1: {'C': [100000.0], 'gamma': [0.1], 'scores': [0.9275, 0.94225, 0.978]}, 2: {'C': [100000.0], 'gamma': [0.1], 'scores': [0.9275, 0.94225, 0.978]}, 3: {'C': [100000.0], 'gamma': [0.1], 'scores': [0.9275, 0.94225, 0.978]}, 4: {'C': [100000.0], 'gamma': [0.1], 'scores': [0.9275, 0.94225, 0.978]}}
